# VacationPy
---

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [14]:
# Load the CSV file created in WeatherPy into a DataFrame
city_data_df = pd.read_csv("../Output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Howrah,22.5892,88.3103,28.00,89,40,2.06,IN,1690319707
1,1,Albany,42.6001,-73.9662,26.46,49,100,1.93,US,1690319475
2,2,Geraldton,-28.7667,114.6000,12.23,76,75,4.12,AU,1690319594
3,3,Puerto Aisén,-45.4000,-72.7000,5.13,100,99,1.10,CL,1690319717
4,4,Lā‘ie,21.6477,-157.9253,30.98,71,66,7.18,US,1690319617


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame with the size of the point being the humidity.

In [40]:


# Configure the map plot
all_cities_plot = city_data_df.hvplot.points(
   "Long",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.75,
    color = "Max Temp",
    clabel = "Temperature (C)",
    hover_cols = ["City", "Country", "Cloudiness"]
    )

# Display the map
all_cities_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Long,Lat]   (Max Temp,Humidity,City,Country,Cloudiness)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [90]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal vacation is somewhere hot, where I can swim outside and enjoy the sun. 
vacation_df = city_data_df.loc[(city_data_df["Max Temp"]>29)&(city_data_df["Cloudiness"] <25)&(city_data_df["Wind Speed"] < 2)]

# Drop any rows with null values
vacation_clean = vacation_df.dropna()

# Display sample data
vacation_clean

,City_ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,Newman,37.3138,-121.0208,36.19,9,1,1.29,US,1690319624
143,143,Pontes e Lacerda,-15.2261,-59.3353,32.54,23,0,1.67,BR,1690319802
216,216,Greeley,40.4233,-104.7091,38.15,25,20,0.00,US,1690319239
230,230,Ascension,30.2169,-90.9001,36.15,38,0,1.54,US,1690319869
237,237,Manicoré,-5.8092,-61.3003,33.97,38,0,1.03,BR,1690319873
436,436,Hassi Messaoud,31.6804,6.0729,39.92,9,0,1.54,DZ,1690319999
461,461,Plácido de Castro,-10.3353,-67.1856,35.26,24,1,0.89,BR,1690320015
528,528,Atmore,31.0238,-87.4939,33.83,48,1,1.18,US,1690320064
583,583,Palmas,-10.2128,-48.3603,29.93,48,0,1.54,BR,1690320109
585,585,Metlili Chaamba,32.2667,3.6333,36.52,9,0,0.00,DZ,1690320110


### Step 3: Create a new DataFrame called `hotel_df`.

In [91]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_clean[["City", "Country", "Lat", "Long", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Long,Humidity,Hotel Name
102,Newman,US,37.3138,-121.0208,9,
143,Pontes e Lacerda,BR,-15.2261,-59.3353,23,
216,Greeley,US,40.4233,-104.7091,25,
230,Ascension,US,30.2169,-90.9001,38,
237,Manicoré,BR,-5.8092,-61.3003,38,
436,Hassi Messaoud,DZ,31.6804,6.0729,9,
461,Plácido de Castro,BR,-10.3353,-67.1856,24,
528,Atmore,US,31.0238,-87.4939,48,
583,Palmas,BR,-10.2128,-48.3603,48,
585,Metlili Chaamba,DZ,32.2667,3.6333,9,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [92]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
bias = "proximity:"
filter = "circle:"

params = {
    "categories": categories,
    "bias": bias,
    "filter": filter,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long = hotel_df.loc[index, "Long"]
    lat = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Newman - nearest hotel: No hotel found
Pontes e Lacerda - nearest hotel: Hotel Avenida
Greeley - nearest hotel: DoubleTree by Hilton Greeley at Lincoln Park
Ascension - nearest hotel: Cajun Country Inn
Manicoré - nearest hotel: No hotel found
Hassi Messaoud - nearest hotel: AMC
Plácido de Castro - nearest hotel: No hotel found
Atmore - nearest hotel: Fairfield Inn & Suites
Palmas - nearest hotel: Hotel Triângulo Mineiro
Metlili Chaamba - nearest hotel: No hotel found
Altamira - nearest hotel: Hotel Vale do Xingu
Hays - nearest hotel: Fall Creek Inn


,City,Country,Lat,Long,Humidity,Hotel Name
102,Newman,US,37.3138,-121.0208,9,No hotel found
143,Pontes e Lacerda,BR,-15.2261,-59.3353,23,Hotel Avenida
216,Greeley,US,40.4233,-104.7091,25,DoubleTree by Hilton Greeley at Lincoln Park
230,Ascension,US,30.2169,-90.9001,38,Cajun Country Inn
237,Manicoré,BR,-5.8092,-61.3003,38,No hotel found
436,Hassi Messaoud,DZ,31.6804,6.0729,9,AMC
461,Plácido de Castro,BR,-10.3353,-67.1856,24,No hotel found
528,Atmore,US,31.0238,-87.4939,48,Fairfield Inn & Suites
583,Palmas,BR,-10.2128,-48.3603,48,Hotel Triângulo Mineiro
585,Metlili Chaamba,DZ,32.2667,3.6333,9,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [93]:


# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
   "Long",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
    )

# Display the map
hotels_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Long,Lat]   (City,Country,Hotel Name)